In [1]:
import sys
sys.path.append('/source/main')
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import pandas as pd

from naruto_skills.voc import Voc

In [9]:
df = pd.read_csv('/source/main/data_for_train/output/my_train.csv')

In [10]:
df.shape

(1531381, 2)

In [11]:
df.dropna(inplace=True)

In [12]:
df.sample(10)

,tgt,src
961705,Polyalthia obliqua là loài thực vật có hoa thu...,Polyalthia obliqua la loai thuc vat co hoa thu...
677730,Tốt cho sức khỏe tim mạch Nhiều nghiên cứu đã ...,Tot cho suc khoe tim mach Nhieu nghien cuu da ...
1140504,"Còn nhớ , thời Huỳnh Đức về đây làm việc , đội...","Con nho , thoi Huynh Duc ve day lam viec , doi..."
1056993,Silene ruinarum là loài thực vật có hoa thuộc ...,Silene ruinarum la loai thuc vat co hoa thuoc ...
3861,Cũng chỉ khoảng 30 % dân số nước này đang sử d...,Cung chi khoang 30 % dan so nuoc nay dang su d...
1342372,Ông qua đời vào ngày 23 tháng 1 năm 1981 tại T...,Ong qua doi vao ngay 23 thang 1 nam 1981 tai T...
462891,Verbesina holwayi là một loài thực vật có hoa ...,Verbesina holwayi la mot loai thuc vat co hoa ...
800750,Indigofera declinata là một loài thực vật có h...,Indigofera declinata la mot loai thuc vat co h...
1493962,Việc tái cấu trúc nguồn lực này sẽ mang lại ch...,Viec tai cau truc nguon luc nay se mang lai ch...
1246395,Hầu hết người dùng không biết về IoT cũng như ...,Hau het nguoi dung khong biet ve IoT cung nhu ...


# Vocab for src

In [13]:
voc = Voc(tokenize_func=Voc.WORD_LV_TOK_FUNC, space_char=Voc.WORD_LV_SPACE_CHR)

In [14]:
voc = Voc(tokenize_func=Voc.WORD_LV_TOK_FUNC, space_char=Voc.WORD_LV_SPACE_CHR)
for doc in df['src']:
    voc.add_sentence(doc)

In [15]:
len(voc.word2count)

350505

## Trim out infrequent vocabs

In [16]:
freq = [(k, v) for k, v in voc.word2count.items() ]
freq.sort(key=lambda x: x[1], reverse=True)

In [17]:
# freq[30000:30100]

In [18]:
voc.trim(min_count=10)

INFO:root:keep_words 33462 / 350505 = 0.0955


In [19]:
voc.build_from_scratch(special_tokens=['__p__', '__o__', '__s__', '__e__'])

INFO:root:Indexing vocabs successfully. Total vocabs: 33466


## Clean vocab

In [23]:
len(voc.word2index)

33464

In [24]:
def is_alpha(tok):
    if len(tok) == 1:
        return True
    return all([c.isalpha() for c in tok])

def is_valid_len(tok):
    return len(tok) <= 10

def is_valid_token(tok):
    if not is_alpha(tok):
        return False
    if not is_valid_len(tok):
        return False
    return True

In [25]:
vocabs = [vocab for vocab in voc.word2index if is_valid_token(vocab)]

In [26]:
len(vocabs)

25386

In [27]:
del voc

In [28]:
voc = Voc(tokenize_func=Voc.WORD_LV_TOK_FUNC, space_char=Voc.WORD_LV_SPACE_CHR)
voc.add_sentence(' '.join(vocabs))
voc.build_from_scratch(special_tokens=['__p__', '__o__', '__s__', '__e__'])

INFO:root:Indexing vocabs successfully. Total vocabs: 25390


In [29]:
len(voc.word2index)

25390

In [34]:
voc.word2index['__o__']

1

In [35]:
voc.dump('/source/main/vocab/output/src.pkl')
print('Done')
del voc

Done


# Vocab for tgt

In [36]:
voc = Voc(tokenize_func=Voc.WORD_LV_TOK_FUNC, space_char=Voc.WORD_LV_SPACE_CHR)
for doc in df['tgt']:
    voc.add_sentence(doc)

In [37]:
len(voc.word2count)

368671

## Trim out infrequent vocabs

In [38]:
freq = [(k, v) for k, v in voc.word2count.items() ]
freq.sort(key=lambda x: x[1], reverse=True)

In [39]:
# freq[60000:60100]

In [40]:
voc.trim(5)

INFO:root:keep_words 65660 / 368671 = 0.1781


In [41]:
voc.build_from_scratch(special_tokens=['__p__', '__o__', '__s__', '__e__'])

INFO:root:Indexing vocabs successfully. Total vocabs: 65664


## Clean vocab

In [42]:
len(voc.word2index)

65664

In [43]:
def is_alpha(tok):
    if len(tok) == 1:
        return True
    return all([c.isalpha() for c in tok])

def is_valid_len(tok):
    return len(tok) <= 10

def is_valid_token(tok):
    if not is_alpha(tok):
        return False
    if not is_valid_len(tok):
        return False
    return True

In [44]:
vocabs = [vocab for vocab in voc.word2index if is_valid_token(vocab)]

In [45]:
len(vocabs)

50433

In [46]:
del voc

In [47]:
voc = Voc(tokenize_func=Voc.WORD_LV_TOK_FUNC, space_char=Voc.WORD_LV_SPACE_CHR)
voc.add_sentence(' '.join(vocabs))
voc.build_from_scratch(special_tokens=['__p__', '__o__', '__s__', '__e__'])

INFO:root:Indexing vocabs successfully. Total vocabs: 50437


In [48]:
len(voc.word2index)

50437

In [49]:
len(voc.index2word)

50437

In [50]:
voc.word2index['.']

5

In [51]:
voc.dump('/source/main/vocab/output/tgt.pkl')
print('Done')
del voc

Done


In [3]:
voc = Voc.load('/source/main/vocab/output/src.pkl')

INFO:root:Indexing vocabs successfully. Total vocabs: 25390


In [4]:
voc.word2index['__s__']

2

In [5]:
voc.word2index['__e__']

3